In [1]:
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1,2"
sys.path.append('/root/code')

from definitions import LOG_DIR, WEIGHT_DIR
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
import time
import datetime
import numpy
from utils.contrastiveLoss import ContrastiveLoss
from models.SCNN18 import SCNN18
from models.SCNN18_random_aug import SCNN18_random_aug
from models.SCNN18_Sigmoid_random_aug import SCNN18_Sigmoid_random_aug
import utils.dataset as dataset
import logging
from logging import handlers

In [2]:
LOG = logging.getLogger('root')

def initLog(debug=False):
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%d %H:%M',
        handlers=[logging.StreamHandler(), handlers.RotatingFileHandler('simclr_0.1s_audio.log', "w", 1024 * 1024 * 100, 3, "utf-8")]
    )
    LOG.setLevel(logging.DEBUG if debug else logging.INFO)
    tf.get_logger().setLevel('ERROR')

In [3]:
initLog()

In [4]:
def get_optimizer(optimizer, lr):
    optimizer = optimizer.lower()
    if optimizer == 'adadelta':
        return tf.optimizers.Adadelta() if lr == 0 else tf.optimizers.Adadelta(learning_rate=lr)
    elif optimizer == 'adagrad':
        return tf.optimizers.Adagrad() if lr == 0 else tf.optimizers.Adagrad(learning_rate=lr)
    elif optimizer == 'adam':
        return tf.optimizers.Adam() if lr == 0 else tf.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'adamax':
        return tf.optimizers.Adamax() if lr == 0 else tf.optimizers.Adamax(learning_rate=lr)
    elif optimizer == 'sgd':
        return tf.optimizers.SGD() if lr == 0 else tf.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop':
        return tf.optimizers.RMSprop() if lr == 0 else tf.optimizers.RMSprop(learning_rate=lr)

In [5]:
# Define model architecture & define hyper-parameter
lr = 1e-4
max_epochs = 360
sample_size=[32000, 1]
batch_size = 32
input_shape = tuple(sample_size)
output_shape = 2
epoch_loss_values = []
step_loss_values = []
epoch_cl_loss_values = []
epoch_recon_loss_values = []

In [6]:
def generate_batched_data(ds_path):
    for index, ds in enumerate(ds_path):
        if index == 0:
            train_ds = dataset.get_dataset_without_label(ds)
            slices_ds = tf.data.Dataset.from_tensor_slices((train_ds))
        else:
            train_ds = dataset.get_dataset_without_label(ds)
            add_ds = tf.data.Dataset.from_tensor_slices((train_ds))
            slices_ds.concatenate(add_ds)
    dataset_length = [i for i, _ in enumerate(slices_ds)][-1] + 1

    return slices_ds.shuffle(dataset_length, seed=8, reshuffle_each_iteration=True).batch(batch_size)

In [7]:
def generate_ground_truth_batched_data(ds_path):
    for index, ds in enumerate(ds_path):
        if index == 0:
            train_ds = dataset.get_ground_truth(ds)
            slices_ds = tf.data.Dataset.from_tensor_slices((train_ds))
        else:
            train_ds = dataset.get_ground_truth(ds)
            add_ds = tf.data.Dataset.from_tensor_slices((train_ds))
            slices_ds.concatenate(add_ds)
    dataset_length = [i for i, _ in enumerate(slices_ds)][-1] + 1

    return dataset_length, slices_ds.shuffle(dataset_length, seed=8, reshuffle_each_iteration=True).batch(batch_size)

In [8]:
# generate batched data
noise_path = ['./SCNN18_0.1second/SCNN-Jamendo-FMA-train_30s-noise.h5']
pitch_path = ['./SCNN18_0.1second/SCNN-Jamendo-FMA-train_30s-pitch.h5']
ground_path = ['./SCNN18_0.1second/SCNN-Jamendo-FMA-train_30s.h5']

In [9]:
noise_ds = generate_batched_data(noise_path)
pitch_ds = generate_batched_data(pitch_path)
ds_length, gt_ds = generate_ground_truth_batched_data(ground_path)

In [10]:
strategy = tf.distribute.MirroredStrategy(devices=[f'/gpu:{i}' for i in range(3)])
with strategy.scope():
    optimizer = get_optimizer('adam', lr)
    model = SCNN18_random_aug(input_shape, output_shape).model()
    # Compile the model
    # contrastive_loss = ContrastiveLoss(temperature=0.05, reduction = tf.keras.losses.Reduction.SUM)
    bin_loss = tf.keras.losses.CategoricalCrossentropy(reduction = tf.keras.losses.Reduction.NONE)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')
    model.summary(print_fn=LOG.info)

    # Define the metrics
    train_loss_metric = tf.keras.metrics.Mean()
    val_loss_metric = tf.keras.metrics.Mean()
    cl_loss_metric = tf.keras.metrics.Mean()
    recon_loss_metric = tf.keras.metrics.Mean()

2024-06-07 05:35 INFO Model: "functional_1"
2024-06-07 05:35 INFO _________________________________________________________________
2024-06-07 05:35 INFO Layer (type)                 Output Shape              Param #   
2024-06-07 05:35 INFO =================================================================
2024-06-07 05:35 INFO input (InputLayer)           [(None, 32000, 1)]        0         
2024-06-07 05:35 INFO _________________________________________________________________
2024-06-07 05:35 INFO STSA (STSA)                  (None, 63, 1024, 1)       4194304   
2024-06-07 05:35 INFO _________________________________________________________________
2024-06-07 05:35 INFO random_zoom (RandomZoom)     (None, 63, 1024, 1)       0         
2024-06-07 05:35 INFO _________________________________________________________________
2024-06-07 05:35 INFO uscl_conv1 (USCLLayer)       (None, 21, 512, 64)       704       
2024-06-07 05:35 INFO ______________________________________________________

In [11]:
def train_step(inputs, inputs_2, gt_input):
    with tf.GradientTape() as tape:
        # Forward pass
        outputs_v1 = model(inputs)
        outputs_v2 = model(inputs_2)
        # flat_out_v1 = tf.reshape(outputs_v1, (batch_size, -1))
        # flat_out_v2 = tf.reshape(outputs_v2, (batch_size, -1))

        # Compute the losses
        # r_loss = tf.reduce_mean(keras.losses.CategoricalCrossentropy(outputs_v1, gt_input))
        r_loss = K.mean(tf.keras.losses.mean_squared_error(gt_input, outputs_v1))
        cl_loss = bin_loss(outputs_v1, outputs_v2)

        # Adjust the CL loss by Recon Loss
        total_loss = r_loss + cl_loss * r_loss

        # Compute the gradients and apply them
        grads = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Update the metrics
    train_loss_metric.update_state(total_loss)
    cl_loss_metric.update_state(cl_loss)
    recon_loss_metric.update_state(r_loss)

In [12]:
# train
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    epoch_loss = 0
    epoch_cl_loss = 0
    epoch_recon_loss = 0
    step = 0

    # Reset the metrics at the start of each epoch
    train_loss_metric.reset_states()
    cl_loss_metric.reset_states()
    recon_loss_metric.reset_states()

    # Train the model
    for n_batch, p_batch, gt_batch in zip(noise_ds, pitch_ds, gt_ds):
        step += 1
        start_time = time.time()

        inputs, inputs_2, gt_input = (
            tf.convert_to_tensor(n_batch, dtype=tf.float32),
            tf.convert_to_tensor(p_batch, dtype=tf.float32),
            tf.convert_to_tensor(gt_batch, dtype=tf.float32),
        )
        strategy.run(train_step, args=(inputs, inputs_2, gt_input))


        epoch_loss += train_loss_metric.result().numpy()
        step_loss_values.append(train_loss_metric.result().numpy())

        # CL & Recon Loss Storage of Value
        epoch_cl_loss += cl_loss_metric.result().numpy()
        epoch_recon_loss += recon_loss_metric.result().numpy()

        end_time = time.time()
        if step % 10 == 0:
            print(
                f"{step}/{ds_length // batch_size}, "
                f"train_loss: {train_loss_metric.result().numpy():.6f}, "
                f"time taken: {end_time-start_time}s"
            )

    # if (epoch + 1) % 5 == 0:
    #     print("Entering Validation for epoch: {}".format(epoch + 1))
    #     valstep = 0
    #     for n_batch, p_batch, gt_batch in zip(val_noise_ds, val_pitch_ds, val_gt_ds):
    #         valstep += 1
    #         start_time = time.time()

    #         inputs, inputs_2, gt_input = (
    #             tf.convert_to_tensor(n_batch, dtype=tf.float32),
    #             tf.convert_to_tensor(p_batch, dtype=tf.float32),
    #             tf.convert_to_tensor(gt_batch, dtype=tf.float32),
    #         )
    #         strategy.run(val_step, args=(inputs, inputs_2, gt_input))
    #     end_time = time.time()
    #     LOG.info(f"epoch {epoch + 1} Validation avg loss: {val_loss_metric.result().numpy():.4f}, " f"time taken: {end_time-start_time}s")
    #     val_loss_metric.reset_states()

    epoch_loss /= step
    epoch_cl_loss /= step
    epoch_recon_loss /= step

    epoch_loss_values.append(epoch_loss)
    epoch_cl_loss_values.append(epoch_cl_loss)
    epoch_recon_loss_values.append(epoch_recon_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
    LOG.info(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

# model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SIMCLR_SCNN_pretrain.h5"))
model.save_weights(os.path.join(WEIGHT_DIR, f"{str(datetime.date.today())}_SIMCLR_SCNN_Jamendo_FMA_30s_pretrain.h5"))

----------
epoch 1/360
10/145, train_loss: 0.411871, time taken: 0.8815691471099854s
20/145, train_loss: 0.402628, time taken: 0.8965892791748047s
30/145, train_loss: 0.398952, time taken: 0.8924219608306885s
40/145, train_loss: 0.385682, time taken: 0.8742778301239014s
50/145, train_loss: 0.381934, time taken: 0.9114646911621094s
60/145, train_loss: 0.377264, time taken: 0.8659765720367432s
70/145, train_loss: 0.379949, time taken: 0.8959596157073975s
80/145, train_loss: 0.375492, time taken: 0.8858809471130371s
90/145, train_loss: 0.370156, time taken: 0.8871195316314697s
100/145, train_loss: 0.371753, time taken: 0.8858199119567871s
110/145, train_loss: 0.372643, time taken: 0.8858203887939453s
120/145, train_loss: 0.372052, time taken: 0.882828950881958s
130/145, train_loss: 0.367796, time taken: 0.907569408416748s
140/145, train_loss: 0.368171, time taken: 0.9283678531646729s


2024-06-07 05:38 INFO epoch 1 average loss: 0.3822


epoch 1 average loss: 0.3822
----------
epoch 2/360
10/145, train_loss: 0.360794, time taken: 0.913261890411377s
20/145, train_loss: 0.354000, time taken: 0.9114460945129395s
30/145, train_loss: 0.351119, time taken: 0.9151489734649658s
40/145, train_loss: 0.347997, time taken: 0.8862833976745605s
50/145, train_loss: 0.352867, time taken: 0.897536039352417s
60/145, train_loss: 0.352547, time taken: 0.9375255107879639s
70/145, train_loss: 0.354447, time taken: 0.9272825717926025s
80/145, train_loss: 0.350443, time taken: 0.9064667224884033s
90/145, train_loss: 0.351276, time taken: 0.8924190998077393s
100/145, train_loss: 0.351034, time taken: 0.9265968799591064s
110/145, train_loss: 0.352292, time taken: 0.9128398895263672s
120/145, train_loss: 0.350246, time taken: 0.9425063133239746s
130/145, train_loss: 0.346880, time taken: 0.9128317832946777s
140/145, train_loss: 0.347714, time taken: 1.0027515888214111s


2024-06-07 05:40 INFO epoch 2 average loss: 0.3506


epoch 2 average loss: 0.3506
----------
epoch 3/360
10/145, train_loss: 0.323580, time taken: 0.9138312339782715s
20/145, train_loss: 0.351048, time taken: 0.9151952266693115s
30/145, train_loss: 0.332974, time taken: 0.9036111831665039s
40/145, train_loss: 0.333961, time taken: 0.9249660968780518s
50/145, train_loss: 0.330943, time taken: 0.9121983051300049s
60/145, train_loss: 0.324808, time taken: 0.9372608661651611s
70/145, train_loss: 0.321483, time taken: 0.9304149150848389s
80/145, train_loss: 0.317167, time taken: 0.9176435470581055s
90/145, train_loss: 0.311509, time taken: 0.9556434154510498s
100/145, train_loss: 0.308932, time taken: 0.9268965721130371s
110/145, train_loss: 0.306103, time taken: 0.9277541637420654s
120/145, train_loss: 0.301038, time taken: 0.9264254570007324s
130/145, train_loss: 0.298009, time taken: 0.9119002819061279s
140/145, train_loss: 0.295584, time taken: 0.945716142654419s


...

epoch 358 average loss: 0.0047
----------
epoch 359/360
10/145, train_loss: 0.002987, time taken: 0.9015119075775146s
20/145, train_loss: 0.005488, time taken: 0.8905935287475586s
30/145, train_loss: 0.007657, time taken: 0.8793292045593262s
40/145, train_loss: 0.008166, time taken: 0.9157619476318359s
50/145, train_loss: 0.008115, time taken: 0.8903489112854004s
60/145, train_loss: 0.007210, time taken: 0.9057607650756836s
70/145, train_loss: 0.008912, time taken: 0.9029715061187744s
80/145, train_loss: 0.008764, time taken: 0.8996989727020264s
90/145, train_loss: 0.008408, time taken: 0.9028434753417969s
100/145, train_loss: 0.008681, time taken: 0.891516923904419s
110/145, train_loss: 0.008276, time taken: 0.87701416015625s
120/145, train_loss: 0.008039, time taken: 0.890388011932373s
130/145, train_loss: 0.008039, time taken: 0.9047825336456299s
140/145, train_loss: 0.008538, time taken: 0.8928039073944092s


2024-06-07 18:54 INFO epoch 359 average loss: 0.0075


epoch 359 average loss: 0.0075
----------
epoch 360/360
10/145, train_loss: 0.004778, time taken: 0.9081461429595947s
20/145, train_loss: 0.004076, time taken: 0.8833141326904297s
30/145, train_loss: 0.003334, time taken: 0.8890707492828369s
40/145, train_loss: 0.003659, time taken: 0.8996446132659912s
50/145, train_loss: 0.004247, time taken: 0.8957943916320801s
60/145, train_loss: 0.004922, time taken: 0.9071390628814697s
70/145, train_loss: 0.005823, time taken: 0.9035732746124268s
80/145, train_loss: 0.006750, time taken: 0.8840358257293701s
90/145, train_loss: 0.006358, time taken: 0.914196252822876s
100/145, train_loss: 0.006090, time taken: 0.85787034034729s
110/145, train_loss: 0.005902, time taken: 0.8822014331817627s
120/145, train_loss: 0.005713, time taken: 0.8860993385314941s
130/145, train_loss: 0.005587, time taken: 0.9071855545043945s
140/145, train_loss: 0.005498, time taken: 0.8959920406341553s


2024-06-07 18:56 INFO epoch 360 average loss: 0.0055


epoch 360 average loss: 0.0055
